### CARLA Offline Inference

This notebook shows how to load model, configuration and data and do inference.

##### Prerequisites
- [Model checkpoints.](https://github.com/ln2697/lead?tab=readme-ov-file#quick-start)
- [Prepared data.](https://ln2697.github.io/lead/docs/carla_training.html#prepare-data)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext ipython_beartype
%beartype

import os

import matplotlib.pyplot as plt
import torch

from lead.data_loader.carla_dataset import CARLAData
from lead.inference.config_open_loop import OpenLoopConfig
from lead.inference.open_loop_inference import OpenLoopInference
from lead.training.config_training import TrainingConfig
from lead.visualization.visualizer import Visualizer

Load the configuration and model

In [ ]:
checkpoint_dir = "outputs/checkpoints/tfv6_resnet34"

open_loop_config = OpenLoopConfig()
training_config = TrainingConfig.load_from_file(
    os.path.join(checkpoint_dir, "config.json")
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

open_loop_inference = OpenLoopInference(
    config_training=training_config,
    config_open_loop=open_loop_config,
    model_path=checkpoint_dir,
    device=device,
)

Setup dataloader on all scenarios

In [ ]:
training_config.carla_root = "data/carla_leaderboard2"

dataset = CARLAData(
    root=training_config.carla_data, config=training_config, random=False
)

Visualize one random sample

In [ ]:
index = 42  # You can change this index to test different samples

sample = dataset[index]
sample = torch.utils.data._utils.collate.default_collate(
    [sample]
)  # Collate into a batch of size 1

visualizer = Visualizer(config=training_config, data=sample, prediction=None)
image = visualizer.visualize_training_labels()

plt.figure(figsize=(20, 10))
plt.imshow(image)
plt.show()

Inference on one random sample

In [ ]:
prediction = open_loop_inference.forward(sample)

Visualize prediction

In [ ]:
visualizer = Visualizer(config=training_config, data=sample, prediction=prediction)
image = visualizer.visualize_training_prediction()

plt.figure(figsize=(20, 10))
plt.imshow(image)
plt.show()